[ 임소영 ] 의사결정트리, 부스팅

[ 피쳐-(임시) ] 단백질(g),나트륨(mg) ,당류(g), 포화지방(g)

[ 데이터셋 ] 
- 맥도날드 : https://www.kaggle.com/datasets/mcdonalds/nutrition-facts
- 롯데리아 : https://www.lotteeatz.com/upload/stg/etc/ria/items.html
- 미국 전 지점 : https://www.kaggle.com/datasets/ulrikthygepedersen/fastfood-nutrition
- 버거킹 : https://emilysinglelife.tistory.com/62
- 맘스터치(이미지) : https://www.momstouch.co.kr/m/brand/notice-view.php?idx=49
- 노브랜드 : https://realjace.tistory.com/entry/%EB%85%B8%EB%B8%8C%EB%9E%9C%EB%93%9C%EB%B2%84%EA%B1%B0-%EB%A9%94%EB%89%B4%EB%B3%84-%EC%B9%BC%EB%A1%9C%EB%A6%AC-%EC%98%81%EC%96%91%EC%84%B1%EB%B6%84-%EC%B4%9D%EC%A0%95%EB%A6%AC
- 프랭크 버거 : https://rooftoper.tistory.com/entry/%ED%94%84%EB%9E%AD%ED%81%AC-%EB%B2%84%EA%B1%B0-%EC%98%81%EC%96%91%EC%84%B1%EB%B6%84

## 자료 준비, 전처리 (결측치, 중복값, 이상치)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
file = './Hamburger.csv'
data = pd.read_csv(file)
data.info()

In [ ]:
data.isna().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data

In [ ]:
plt.boxplot(data[['calories','sodium','sugar','total_fat','protein']])
plt.show()

In [ ]:
def print_flier(insert_data):
    q1 = insert_data.quantile(0.25)
    q3 = insert_data.quantile(0.75)
    iqr = q3 - q1
    #print(f"[ q1 ] \n{q1}\n\n[ q3 ] \n{q3}\n")
    #print(f"[ iqr의 범위 ]\n{iqr}\n")

    print('------------------ 이상값이 될 기준 계산 ------------------\n')

    lower = q1 - 1.5 * iqr
    #print(f"[ lower의 값 ]\n{lower}\n")
    upper = q3 + 1.5 * iqr
    #print(f"[ upper의 값 ]\n{upper}\n")

    mask = insert_data < lower
    print(f"[ lower의 개수 ]\n{mask.sum()}\n")
    mask2 = insert_data > upper
    print(f"[ upper의 개수 ]\n{mask2.sum()}\n")

    return lower, upper

lower, upper = print_flier(data[['calories','sodium','sugar','total_fat','protein']])

In [ ]:
data[['calories','sodium','sugar','total_fat','protein']].shape

In [ ]:
def drop_flier(insert_data, col_name):
    global lower
    global upper
    print(f"이상치 제거 전 shape 확인: {insert_data.shape}")
    mask = insert_data[col_name] >= lower.loc[col_name]
    insert_data = insert_data[mask]
    #print(f" '{col_name}' 컬럼에서 lower 미만의 값 제거 후 shape: {insert_data.shape}")

    mask2 = insert_data[col_name] <= upper.loc[col_name]
    insert_data = insert_data[mask2]
    #print(f" '{col_name}' 컬럼에서 upper 초과의 값 제거 후 shape: {insert_data.shape}\n")
    
    print(f"이상치 제거 후 shape 확인: {insert_data.shape} \n")
    return insert_data

data_for_drop = data[['calories','sodium','sugar','total_fat','protein']]
# data_for_drop = drop_flier(data_for_drop, 'calories')
# data_for_drop = drop_flier(data_for_drop, 'sodium')
# data_for_drop = drop_flier(data_for_drop, 'sugar')
# data_for_drop = drop_flier(data_for_drop, 'total_fat')
# data_for_drop = drop_flier(data_for_drop, 'protein')


# 이상치 제거한 데이터에 맞게 원본 데이터도 제거된 인덱스 번호를 같이 drop 시켜줌
# for i in list(data.index):
#     if i not in (list(data_for_drop.index)):
#         data.drop(i,inplace = True)
# print(data.shape)
# 
# # 인덱스 번호가 서로 일치하는지 확인
# print(len(list(data_for_drop.index)) == len(list(data.index)))
# # 서로 일치하는 것을 확인 했으니 이 데이터를 이용해서 알고리즘 돌리겠다.

이상치를 제거했더니 과대 적합이 계속 나왔다. 이상치를 다시 복구 했더니 과대적합이 덜나와서 이상치를 다시 복구했다.

In [ ]:
data.info()

In [ ]:
data.corr(numeric_only=True)

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.scatter(data['calories'], data['sodium'], label = 'sodium', c = 'red', alpha = 0.7)
plt.legend(loc = 'best')
plt.xlabel('calories')
plt.title("[visualization with calories]")

plt.subplot(1,2,2)
plt.scatter(data['calories'], data['sugar'], label = 'sugar', alpha = 0.7)
plt.scatter(data['calories'], data['total_fat'], label = 'total_fat', alpha = 0.7)
plt.scatter(data['calories'], data['protein'], label = 'protein', alpha = 0.7)
plt.legend(loc = 'best')
plt.xlabel('calories')
plt.title("[visualization with calories]")
plt.show()

## 데이터셋 분리 & 스케일링 & 학습 & 평가

In [ ]:
target = data['calories']
feature = data[['sodium','sugar','total_fat','protein']]

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
train_score_list = []
test_score_list = []

# 최고의 random_state 값 찾기
'''
for number in range(1,1001):
    X_train, X_test, y_train, y_test = train_test_split(feature, target, random_state = number)
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    scaled_X_train = scaler.transform(X_train)
    scaled_X_test = scaler.transform(X_test)
    X_train = pd.DataFrame(scaled_X_train, columns = ['sodium', 'sugar', 'total_fat', 'protein'])
    X_test = pd.DataFrame(scaled_X_test, columns = ['sodium', 'sugar', 'total_fat', 'protein']) 
    
    model = DecisionTreeRegressor()
    model.fit(X_train, y_train)
    
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    train_score_list.append(train_score)
    test_score_list.append(test_score)
    #print(f"{number} => [train score] {train_score}      [test_score] {test_score}")
    
max_score = max(test_score_list)
for i in range(len(test_score_list)):
    if test_score_list[i] == max_score:
        print(f"최고의 random_state 값은 {i+1}")
        best_random = i+1
        '''        
# 425

In [ ]:
# 스케일링이 잘 되었는지 확인해보자
"""
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.scatter(y_train, X_train['sodium'], label = 'sodium', alpha = 0.5)
plt.scatter(y_train, X_train['sugar'], label = 'sugar', alpha = 0.5)
plt.scatter(y_train, X_train['total_fat'], label = 'total_fat', alpha = 0.5)
plt.scatter(y_train, X_train['protein'], label = 'protein', alpha = 0.5)
plt.legend(loc = 'best')
plt.title('preprocessing with STD scaler - TRAIN')

plt.subplot(1,2,2)
plt.scatter(y_test, X_test['sodium'], label = 'sodium', alpha = 0.5)
plt.scatter(y_test, X_test['sugar'], label = 'sugar', alpha = 0.5)
plt.scatter(y_test, X_test['total_fat'], label = 'total_fat', alpha = 0.5)
plt.scatter(y_test, X_test['protein'], label = 'protein', alpha = 0.5)
plt.legend(loc = 'best')
plt.title('preprocessing with STD scaler - TEST')

plt.tight_layout
plt.show()
"""

# 스케일링이 잘된 것을 확인했다

## 튜닝 : GridSearchCV 사용

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, random_state = 425)

scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)
X_train = pd.DataFrame(scaled_X_train, columns = ['sodium', 'sugar', 'total_fat', 'protein'])
X_test = pd.DataFrame(scaled_X_test, columns = ['sodium', 'sugar', 'total_fat', 'protein']) 

In [ ]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)

train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
train_score_list.append(train_score)
test_score_list.append(test_score)
print(f"[train score] {train_score}      [test_score] {test_score}")
# 과대 적합이 나왔다.

In [ ]:
criterion_list = ['squared_error','friedman_mse','absolute_error','poisson']
splitter_list = ['best', 'random']
max_depth_list = [2,3,4,5,6,7,8,9,10]
min_samples_split_list = [1,2,3,4,5,6,7,8,9,10]
min_samples_leaf_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
max_features_list = [1,2,3,4]
min_impurity_decrease_list = [1,2,3,4,5,6,7]

list_for_df = []

for A in criterion_list:
    for B in splitter_list:
        for C in max_depth_list:
            for D in min_samples_split_list:
                for E in min_samples_leaf_list:
                    for F in max_features_list:
                        for G in min_impurity_decrease_list:
                            sample_list = []
                            
                            model = DecisionTreeRegressor()
                            model.fit(X_train, y_train)
                            
                            train_score = model.score(X_train, y_train)
                            test_score = model.score(X_test, y_test)
                            sample_list.append(A)
                            sample_list.append(B)
                            sample_list.append(C)
                            sample_list.append(D)
                            sample_list.append(E)
                            sample_list.append(F)
                            sample_list.append(G)
                            sample_list.append(train_score)
                            sample_list.append(test_score)
                            list_for_df.append(sample_list)
                            
col_list = ['criterion','splitter','max_depth','min_samples_split','min_samples_lear','max_feautures','min_impurity_decrease']                            
result = pd.DataFrame(list_for_df, columns=col_list)
result

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
DTR_model = DecisionTreeRegressor()

param = {'criterion':['squared_error','friedman_mse','absolute_error','poisson'],
         'splitter' : ['best', 'random'],
         'max_depth': [2, 3, 4, 5],
         'min_samples_leaf': [1,2,3,4,5]
        }   
cv_num = 5

# 모델 생성
gs = GridSearchCV(DTR_model, param_grid = param, cv = cv_num, return_train_score = True)

# 학습 및 평가
result = gs.fit(X_train, y_train)
resultDF = pd.DataFrame(gs.cv_results_)
print(f"max train score : {max(resultDF['mean_train_score'])}")
print(f"max test score : {max(resultDF['mean_test_score'])}\n")

# 최고 성능의 파라미터, 최고의 점수
print(f"best parameters : {gs.best_params_}")
print(f"best score : {gs.best_score_}")

In [ ]:
total_model = gs.best_estimator_
total_model

## 예측

In [ ]:
y_pred = total_model.predict(X_test)
y_pred

## 성능 평가

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
print(f"R2 SCORE : {r2_score(y_test, y_pred)}")
print(f"MSE : {mean_squared_error(y_test, y_pred)}")
print(f"MAE : {mean_absolute_error(y_test, y_pred)}")

## 모델 저장

In [ ]:
import joblib
import os

In [ ]:
# model_dir = './YSY_model/'
# model_filename = model_dir + 'decision_tree_03.pkl'
# 
# if not os.path.exists(model_dir):
#     os.mkdir(model_dir)
#     
# joblib.dump(total_model, model_filename)

## decision tree 시각화

In [ ]:
# 생성된 모델 구조 시각화
from sklearn.tree import export_graphviz
import graphviz
import pydot

In [ ]:
total_model.n_features_in_

In [ ]:
total_model.max_features_

In [ ]:
total_model.tree_

In [ ]:
num = 4

export_graphviz(total_model,feature_names = total_model.feature_names_in_, rounded = True, filled = True, out_file = f"hamburger_tree_{num}.dot")

In [ ]:
from graphviz import Source
from graphviz import render
source = Source.from_file(f"hamburger_tree_{num}.dot", format="png")
source.render(f"DecisionTree_{num}", format="png", cleanup=True)
print("저장됨")